In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, , , Waiting, )

In [2]:
import sys  # For simplicity, we'll read config file from 1st CLI param sys.argv[1]
import json
import logging
import requests
import msal
import pandas as pd
from datetime import date, timedelta

# Optional logging
# logging.basicConfig(level=logging.DEBUG)  # Enable DEBUG log for entire script
# logging.getLogger("msal").setLevel(logging.INFO)  # Optionally disable MSAL DEBUG logs

config = {
            "authority": "https://login.microsoftonline.com/ReplaceWithYourTenantId",
            "client_id": "ReplaceWithYourClientId",
            "scope": ["https://analysis.windows.net/powerbi/api/.default"],
            "secret": "ReplaceWithYourClientId'sSecret",
            "endpoint": "https://api.powerbi.com/v1.0/myorg/admin/activityevents"
        }

#Get yesterdays date and convert to string


# Create a preferably long-lived app instance which maintains a token cache.
app = msal.ConfidentialClientApplication(
    config["client_id"], authority=config["authority"],
    client_credential=config["secret"],
    )

# The pattern to acquire a token looks like this.
result = None

# Firstly, looks up a token from cache
# Since we are looking for token for the current app, NOT for an end user,
# notice we give account parameter as None.
result = app.acquire_token_silent(config["scope"], account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scopes=config["scope"])

if "access_token" in result:
    # Calling graph using the access token
   logging.info("Got the Token")

else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id")) 

StatementMeta(, , , Waiting, )

In [9]:
#print(result)
activityDate = date.today() - timedelta(days=1)
activityDate = activityDate.strftime("%Y-%m-%d")
print(activityDate)

#Set Power BI REST API to get Activities for today
url = "https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='" + activityDate + "T00:00:00'&endDateTime='" + activityDate + "T23:59:59'"

access_token = result['access_token']
header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}
response = requests.get(url=url, headers=header)

response_obj = response.json()

event_entities = response_obj["activityEventEntities"]
continuation_uri = response_obj["continuationUri"]
continuation_token = response_obj["continuationToken"]
activity_events = event_entities
cont_count = 1
while continuation_token is not None:
    response = requests.get(continuation_uri, headers=header)
    response_obj = response.json()

    event_entities = response_obj["activityEventEntities"]
    continuation_uri = response_obj["continuationUri"]
    continuation_token = response_obj["continuationToken"]

    activity_events.extend(event_entities)
    cont_count += 1      

print(f"Took {cont_count} tries to exhaust continuation token for {len(activity_events)} events.") 


StatementMeta(, , , Waiting, )

2023-06-01


Took 24 tries to exhaust continuation token for 117110 events.


In [19]:
#df.info()
df = pd.DataFrame(activity_events)
Object_cols =[col for col, col_type in df.dtypes.items() if col_type=="object"]
df[Object_cols] = df[Object_cols].astype(str)

float64_cols =[col for col, col_type in df.dtypes.items() if col_type=="float64"]
df[float64_cols] = df[float64_cols].astype(str)

sdf = spark.createDataFrame(df)

StatementMeta(, , , Waiting, )

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [11]:
from pyspark.sql.functions import col,year,month
table_name="AuditRecords"

sdf = sdf.withColumn('Year', year(col("CreationTime")))
sdf = sdf.withColumn('Month', month(col("CreationTime")))
sdf.write.mode("append").option("mergeSchema","true").format("delta").partitionBy("Year","Month").save("Tables/" + table_name)



StatementMeta(, , , Waiting, )